In [1]:
import math
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from scipy.optimize import minimize
from sklearn import metrics
import time
import pandas as pd

###  a) The Chierici’s relative permeability model 

**Eq. 5**
\begin{equation}
\kappa_{r w}=\kappa_{w}^{0} e^{-B\left(\frac{S_{w}-S_{w 0}}{1-S_{w}-S_{o 0}}\right)^{-M}}
\quad \kappa_{r o}=\kappa_{o}^{0} e^{-A\left(\frac{S_{w}-S_{w 0}}{1-S_{w}-S_{o 0}}\right)^{L}}
\end{equation}

In [2]:
#water relative permeability at the residual oil saturation /// krwe in code water rel perm at water curve end point
kw0=0.375
#oil relative permeability at the connate water saturation /// kroe in code oil rel perm at oil curve end point
ko0=0.95
#𝑆w0  - connate water saturation /// residual_w = residual water saturation, Swirr (irreducible)
sw0=0.2
#𝑆𝑜0 - residual oil saturation/// residual_o =, Sor
so0=0.26
    



**Eq. 5**
\begin{equation}
\kappa_{r w}=\kappa_{w}^{0} e^{-B\left(\frac{S_{w}-S_{w 0}}{1-S_{w}-S_{o 0}}\right)^{-M}}
\quad \kappa_{r o}=\kappa_{o}^{0} e^{-A\left(\frac{S_{w}-S_{w 0}}{1-S_{w}-S_{o 0}}\right)^{L}}
\end{equation}

In [11]:
from scipy import optimize
#initial data
kw0=0.375
ko0=0.95
sw0=0.2
so0=0.26

filepath = '/Users/mikhailgasanov/Desktop/Geostatistics and Reservoir Simulation/Homework/dataset1.xlsx'
qi = pd.read_excel(filepath)

In [12]:
# add random table for predictions
qi['Krw_predicted'] = qi['Krw'] - 0.1

In [13]:
np.sqrt(np.mean(qi['Krw']-qi['Krw_predicted'])**2)

0.10000000000000003

In [14]:
qi

,Date,Sw,Krw,Kro,Krw_predicted
0,1,0.20,0.000,0.950,-0.100
1,2,0.22,0.000,0.941,-0.100
2,3,0.24,0.000,0.917,-0.100
3,4,0.26,0.000,0.879,-0.100
4,5,0.28,0.000,0.828,-0.100
5,6,0.30,0.000,0.766,-0.100
6,7,0.32,0.001,0.696,-0.099
7,8,0.34,0.002,0.621,-0.098
8,9,0.36,0.002,0.545,-0.098
9,10,0.38,0.004,0.470,-0.096


In [20]:
class Project():
    def __init__(self, df):
        self.df = df
        self.kw0=0.375
        self.ko0=0.95
        self.sw0=0.2
        self.so0=0.26
    
    def k_rw(self, parameters):
        #compute k_rw for each sw in dataframe
        k_rw_predicted = []
        B, M = parameters

        k_rw = self.kw0
        for i in range(len(test.df)):
            if i==0:
                _part = (qi.loc[i, 'Sw']-self.sw0)/(1-qi.loc[i, 'Sw']-self.so0)
                k_rw_new = self.kw0 * np.exp((-B*_part)**(-M))
            else: 
                _part = (qi.loc[i, 'Sw']-qi.loc[i-1, 'Sw'])/(1-qi.loc[i, 'Sw']-self.so0)
                k_rw_new = k_rw * np.exp((-B*_part)**(-M))
            k_rw_predicted.append(k_rw_new)
            k_rw = k_rw_new

        rmse = np.sqrt(np.mean(self.df['Krw']-k_rw_predicted)**2)
        print(rmse)
        return rmse
    def optimization(self, B, M):
        x0 = np.array([B,M])
        bnds = ((1.86,2.96), (0.64, 0.97))
        res = optimize.minimize(self.k_rw, x0, bounds=bnds, \
        method='COBYLA')
        return res

In [22]:
# B=2.83
# M=0.64
test = Project(qi)
B, M = 2.83, 0.64
res = test.optimization(B,M)
res

inf
inf
inf


/Users/mikhailgasanov/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:541: RuntimeWarning: Method COBYLA cannot handle bounds.
  warn('Method %s cannot handle bounds.' % method,
<ipython-input-20-e2359d801e01>:18: RuntimeWarning: divide by zero encountered in double_scalars
  k_rw_new = self.kw0 * np.exp((-B*_part)**(-M))
<ipython-input-20-e2359d801e01>:21: RuntimeWarning: invalid value encountered in double_scalars
  k_rw_new = k_rw * np.exp((-B*_part)**(-M))
<ipython-input-20-e2359d801e01>:20: RuntimeWarning: divide by zero encountered in double_scalars
  _part = (qi.loc[i, 'Sw']-qi.loc[i-1, 'Sw'])/(1-qi.loc[i, 'Sw']-self.so0)


     fun: inf
   maxcv: 0.0
 message: 'NaN result encountered.'
    nfev: 3
  status: 5
 success: False
       x: array([2.83, 0.64])

In [23]:
#Trying to compute without class
k_rw_predicted = []
B, M = 2.83, 0.64
k_rw = test.kw0
for i in range(len(test.df)):
    sw = qi.loc[i, 'Sw']
    if i==0:
        k_rw_new = test.kw0 * np.exp((-B*((qi.loc[i, 'Sw']-test.sw0)/(1-qi.loc[i, 'Sw']-test.so0)))**(-M))
    else: 
        _part = (qi.loc[i, 'Sw']-qi.loc[i-1, 'Sw'])/(1-qi.loc[i, 'Sw']-test.so0)
#         print(np.exp(-B*_part**(-M)))
        k_rw_new = k_rw * np.exp(-B*_part**(-M))
    k_rw_predicted.append(k_rw_new)
    k_rw = k_rw_new
#     print(k_rw)
rmse = np.sqrt(np.mean(test.df['Krw']-k_rw_predicted)**2)

<ipython-input-23-32a231bc5eaf>:7: RuntimeWarning: divide by zero encountered in double_scalars
  k_rw_new = test.kw0 * np.exp((-B*((qi.loc[i, 'Sw']-test.sw0)/(1-qi.loc[i, 'Sw']-test.so0)))**(-M))
<ipython-input-23-32a231bc5eaf>:9: RuntimeWarning: divide by zero encountered in double_scalars
  _part = (qi.loc[i, 'Sw']-qi.loc[i-1, 'Sw'])/(1-qi.loc[i, 'Sw']-test.so0)
<ipython-input-23-32a231bc5eaf>:11: RuntimeWarning: invalid value encountered in double_scalars
  k_rw_new = k_rw * np.exp(-B*_part**(-M))


In [24]:
rmse

inf

In [19]:
k_rw * np.exp((-B*((qi.loc[i, 'Sw']-qi.loc[i-1, 'Sw'])/(1-qi.loc[i, 'Sw']-test.so0)))**(-M))

nan